In [15]:
import os
import sys
import glob
from os.path import abspath
os.environ['SPARK_HOME'] = 'C:\\Users\\Chiara\\Documents\\git_projects\\BigData\\spark-3.5.0-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:\\Users\\Chiara\\Documents\\git_projects\\BigData\\spark-3.5.0-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-17.0.3.1'
os.environ['SCALA_HOME'] = 'C:\\AppData\\Local\\Coursier\\data\\bin'

In [16]:
spark_python = os.path.join(os.environ.get('SPARK_HOME',None),'python')
py4j = glob.glob(os.path.join(spark_python,'lib','py4j-*.zip'))[0]
graphf = glob.glob(os.path.join(spark_python,'graphframes.zip'))[0]
sys.path[:0]=[spark_python,py4j]
sys.path[:0]=[spark_python,graphf]
os.environ['PYTHONPATH']=py4j+os.pathsep+graphf

In [17]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Chiara\\Documents\\git_projects\\BigData\\spark-3.5.0-bin-hadoop3'

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark Examples").enableHiveSupport().getOrCreate()


In [19]:
from graphframes import *
import networkx as nx
import matplotlib.pyplot as plt

In [20]:
vertices = spark.createDataFrame([
    ("Alice", 45),
    ("Jacob", 43),
    ("Roy", 21),
    ("Ryan", 49),
    ("Emily", 24),
    ("Sheldon", 52)],
    ["id", "age"]
)
edges = spark.createDataFrame([("Sheldon", "Alice", "Sister"),
                              ("Alice", "Jacob", "Husband"),
                              ("Emily", "Jacob", "Father"),
                              ("Ryan", "Alice", "Friend"),
                              ("Alice", "Emily", "Daughter"),
                              ("Alice", "Roy", "Son"),
                              ("Jacob", "Roy", "Son")],
                             ["src", "dst", "relation"])
vertices.show()
edges.show()

Py4JJavaError: An error occurred while calling o126.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (Turing.fritz.box executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at sca